# Water stress impact factors 

Exiobase categories:
- All that match this regex: 'Water Consumption Blue.*'

LC-Impact stressors
- Water stress

In [3]:
# import required libraries
import pymrio
import numpy as np
import pandas as pd
import pycountry as pyc

In [ ]:
# exiobase 2011 is used for calculating share of stressor for each region-product pair
exio3_11 = pymrio.parse_exiobase3(path="/Users/tterimaa/code/projects/impact-factors/IOT_2011_pxp.zip")
# exiobase 2019 is used for impact factors
exio3_19 = pymrio.parse_exiobase3(path="/Users/tterimaa/code/projects/impact-factors/IOT_2019_pxp")

## Calculate DRorigin
DRorigin is a matrix which describes the amount of the driver of biodiversity loss (DR) that occurs in impact region i sector k and is driven by consumption in region j sector k.

To calculate the matrix we need to
1. Aggregate relevant drivers from exiobase together
2. Diagonalize the aggregated driver and re-calculate the IO system

In [5]:
# aggregate all blue water consumption related drivers
groups = exio3_11.satellite.get_index(as_dict=True, grouping_pattern={"Water Consumption Blue.*": "Water Consumption Blue -– Total"})

exio3_11.satellite_agg = exio3_11.satellite.copy(new_name="Aggregated blue water consumption accounts")

for df_name, df in zip(exio3_11.satellite_agg.get_DataFrame(data=False, with_unit=True, with_population=False),
exio3_11.satellite_agg.get_DataFrame(data=True, with_unit=True, with_population=False)):
    if df_name == "unit":
        exio3_11.satellite_agg.__dict__[df_name] = df.groupby(groups).apply(lambda x: " & ".join(x.unit.unique()))
    else:
        exio3_11.satellite_agg.__dict__[df_name] = df.groupby(groups).sum()

In [ ]:
# diagonalize total blue water consumption and calculate blue water consumption accounts
# optimized for speed
diag_extension = exio3_11.satellite_agg.diag_stressor(("Water Consumption Blue -– Total"))

# First, make sure that L matrix is calculated 
if exio3_11.L is None:
    # try loading the L matrix from pickles/exio3_11_L.pickle
    try:
        exio3_11.L = pd.read_pickle("pickles/exio3_11_L.pickle")
        print("L matrix loaded from pickle")
    except FileNotFoundError:
        print("L matrix not found, calculating it from scratch.")
        print("Calculating A")
        exio3_11.A = pymrio.calc_A(exio3_11.Z, exio3_11.x)
        print("Calculating L")
        exio3_11.L = pymrio.calc_L(exio3_11.A)
        # save the L matrix to a pickle file
        pd.to_pickle(exio3_11.L, "pickles/exio3_11_L.pickle")
else:
    print("L already loaded")

# calculate S (direct emission multipliers)
if diag_extension.S is None:
    print("Calculating S")
    diag_extension.S = pymrio.calc_S(diag_extension.F, exio3_11.x)

Y_agg = exio3_11.Y.groupby(level="region", axis=1, sort=False).sum()

diag_extension.D_cba, _, _, _ = pymrio.calc_accounts(diag_extension.S, exio3_11.L, Y_agg)
diag_extension.D_cba

## Calculate DR share
DR share is a new matrix that represents the share of the driver in the impact region i from the total amount of driver that is driven by consumption in region j sector k.

To calculate the matrix each column of DR origin is shared by the sum of that column.

In [12]:
# calculate dr_s - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k
columns = {}
for series_name, series in diag_extension.D_cba.items():
    series_sum = series.sum()
    columns[series_name] = series / series_sum

dr_s = pd.DataFrame(columns)
dr_s

AT  \
                                                            Paddy rice   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              1.041094e-12   
       Cereal grains nec                                  4.749575e-09   
       Vegetables, fruit, nuts                            1.555138e-09   
       Oil seeds                                          4.325053e-10   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       0.000000e+00   
       Recreational, cultural and sporting services (92)  0.000000e+00   
       Other services (93)                                0.000000e+00   
       Private households with employed persons (95)      0.000000e+00   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                        \
                                                                 Wheat   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              1.901168e-05   
       Cereal grains nec                                  1.004692e-06   
       Vegetables, fruit, nuts                            1.282151e-06   
       Oil seeds                                          2.078883e-07   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       0.000000e+00   
       Recreational, cultural and sporting services (92)  0.000000e+00   
       Other services (93)                                0.000000e+00   
       Private households with employed persons (95)      0.000000e+00   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                            \
                                                         Cereal grains nec   
region sector                                                                
AT     Paddy rice                                             0.000000e+00   
       Wheat                                                  1.139423e-09   
       Cereal grains nec                                      7.615176e-02   
       Vegetables, fruit, nuts                                3.281278e-06   
       Oil seeds                                              4.496833e-07   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)           0.000000e+00   
       Recreational, cultural and sporting services (92)      0.000000e+00   
       Other services (93)                                    0.000000e+00   
       Private households with employed persons (95)          0.000000e+00   
       Extra-territorial organizations and bodies             0.000000e+00   

                                                                                  \
                                                         Vegetables, fruit, nuts   
region sector                                                                      
AT     Paddy rice                                                   0.000000e+00   
       Wheat                                                        4.938578e-10   
       Cereal grains nec                                            1.260616e-06   
       Vegetables, fruit, nuts                                      7.379877e-02   
       Oil seeds                                                    1.572476e-07   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)                 0.000000e+00   
       Recreational, cultural and sporting services (92)     

## Calculate DR unit
DR unit is a region harmonized version of DR share.

To calculate DR unit we need to:
1. Identify regions that are missing from exiobase (rest of the world regions) but are present in lc-impact.
2. Assume that the impacts are divided evenly across the rest of the world category
3. By that assumption we can add the missing countries to DR share as the impact of the row region where country belongs to divided by the number of countries in that region

In [18]:
# harmonize regions 

# for this harmonization we need to know which countries go into:
# 1. rest of the world europe
# 2. rest of the world asia pacific
# 3. rest of the world middle east
# 4. rest of the world africa
# 5. rest of the world americas

row_eu_countries = {
    'AL': 'Albania',
    'AZ': 'Azerbaijan',
    'BA': 'Bosnia and Herzegovina',
    'BY': 'Belarus',
    'FO': 'Faroe Islands',
    'IS': 'Iceland',
    'GL': 'Greenland',
    'GE': 'Georgia',
    'MK': 'Macedonia',
    'MD': 'Moldova',
    'ME': 'Montenegro',
    'RS': 'Serbia',
    'SJ': 'Svalbard',
    'UA': 'Ukraine',
    # overseas territories of EU countries 
    'GP': 'Guadeloupe',
    'PF': 'French Polynesia',
    'FK': 'Falkland Islands (Islas Malvinas)',
    'GF': 'French Guiana',
    'VG': 'Virgin Islands',
    'RE': 'Reunion',
    'VC': 'Saint Vincent and the Grenadines',
}

row_asia_pacific_countries = {
    'BD': 'Bangladesh', 'BN': 'Brunei', 'BT': 'Bhutan', 'CK': 'Cook Islands', 'KH': 'Cambodia', 'FJ': 'Fiji',
    'KZ': 'Kazakhstan', 'KI': 'Kiribati', 'KG': 'Kyrgyzstan', 'LA': 'Laos', 'MY': 'Malaysia',
    'MN': 'Mongolia', 'MM': 'Myanmar (Burma)', 'NP': 'Nepal', 'NC': 'New Caledonia', 'NZ': 'New Zealand',
    'KP': 'North Korea', 'PK': 'Pakistan', 'PG': 'Papua New Guinea', 'PH': 'Philippines',
    'WS': 'Samoa', 'SB': 'Solomon Islands', 'LK': 'Sri Lanka',
    'TJ': 'Tajikistan', 'TH': 'Thailand', 'TM': 'Turkmenistan',
    'UZ': 'Uzbekistan', 'VU': 'Vanuatu', 'VN': 'Vietnam'
}

row_african_countries = {
    'DZ': 'Algeria', 'AO': 'Angola', 'BJ': 'Benin', 'BW': 'Botswana', 'BF': 'Burkina Faso', 'BI': 'Burundi',
    'CM': 'Cameroon', 'CV': 'Cape Verde', 'CF': 'Central African Republic', 'TD': 'Chad', 'KM': 'Comoros',
    'CG': 'Congo', 'CD': 'Congo DRC', 'DJ': 'Djibouti', 'EG': 'Egypt', 'EH': 'Western Sahara', 'GQ': 'Equatorial Guinea', 'ER': 'Eritrea',
    'ET': 'Ethiopia', 'GA': 'Gabon', 'GM': 'Gambia, The', 'GH': 'Ghana', 'GN': 'Guinea', 'GW': 'Guinea-Bissau',
    'CI': 'Ivory Coast', 'KE': 'Kenya', 'LS': 'Lesotho', 'LR': 'Liberia', 'LY': 'Libya', 'MG': 'Madagascar',
    'MW': 'Malawi', 'ML': 'Mali', 'MR': 'Mauritania', 'MU': 'Mauritius', 'MA': 'Morocco', 'MZ': 'Mozambique',
    'NA': 'Namibia', 'NE': 'Niger', 'NG': 'Nigeria', 'RW': 'Rwanda', 'ST': 'São Tomé and Príncipe', 'SN': 'Senegal',
    'SL': 'Sierra Leone', 'SO': 'Somalia', 'SZ': 'Eswatini',
    'SD': 'Sudan', 'TZ': 'Tanzania', 'TG': 'Togo', 'TN': 'Tunisia', 'UG': 'Uganda', 'ZM': 'Zambia', 'ZW': 'Zimbabwe'
}

row_american_countries = {
    'AR': 'Argentina', 'BS': 'Bahamas, The', 'BZ': 'Belize', 'BO': 'Bolivia',
    'CL': 'Chile', 'CO': 'Colombia', 'CR': 'Costa Rica', 'CU': 'Cuba', 'DO': 'Dominican Republic', 'EC': 'Ecuador',
    'SV': 'El Salvador', 'GT': 'Guatemala', 'GY': 'Guyana', 'HT': 'Haiti', 'HN': 'Honduras', 'JM': 'Jamaica',
    'NI': 'Nicaragua', 'PA': 'Panama', 'PY': 'Paraguay', 'PE': 'Peru', 'PR': 'Puerto Rico',
    'SR': 'Suriname', 'TT': 'Trinidad and Tobago', 'UY': 'Uruguay', 'VE': 'Venezuela'
}

row_middle_eastern_countries = {
    'AF': 'Afghanistan', 'AM': 'Armenia',
    'IR': 'Iran', 'IQ': 'Iraq', 'IL': 'Israel', 'JO': 'Jordan', 'KW': 'Kuwait', 'LB': 'Lebanon', 'OM': 'Oman',
    'QA': 'Qatar', 'SA': 'Saudi Arabia', 'SY': 'Syria', 'AE': 'United Arab Emirates',
    'YE': 'Yemen'
}

exio_regions = exio3_11.get_regions()
row_regions = {
    "WA": "Asia and pacific",
    "WE": "Europe",
    "WF": "Africa",
    "WM": "Middle east",
    "WL": "America"
}

# harmonize regions between lc-impact and exiobase
all_row_region_keys = list(row_eu_countries.keys()) + list(row_asia_pacific_countries.keys()) + list(row_african_countries.keys()) + list(row_american_countries.keys()) + list(row_middle_eastern_countries.keys())
exio_regions_without_row = [region for region in exio_regions if region not in row_regions.keys()]
print(len(exio_regions_without_row)) # exiobase contains 44 exact regions + 5 row regions

# combine exact regions from exiobase with exact row regions
all_regions = list(all_row_region_keys) + list(exio_regions_without_row)
print("Regions: ", len(all_regions))
print("Unique regions: ", len(set(all_regions)))
assert len(all_regions) == len(set(all_regions)), "There are duplicate regions in the all_regions list"

44
Regions:  185
Unique regions:  185


In [ ]:
# load and prepare lc-impact data for water consumption
lci_water = pd.read_excel("/Users/tterimaa/code/projects/impact-factors/lc-impact/LC-Impact/12-water consumption/CFs_water_consumption_ecosystems_20180831.xlsx", sheet_name="CF per countries", dtype={1: float, 2: float}, skiprows=2)
lci_water

,Country,CF certain effects [PDF·yr/m3],CF all effects [PDF·yr/m3]
0,Afghanistan,1.566105e-14,1.567708e-14
1,Albania,3.151290e-15,3.239912e-15
2,Algeria,4.926726e-14,7.703947e-14
3,Angola,2.998786e-15,4.276781e-15
4,Argentina,2.518376e-15,2.634408e-15
...,...,...,...
180,Western Sahara,2.013816e-15,2.596501e-15
181,Western Samoa,0.000000e+00,0.000000e+00
182,Yemen,5.808963e-14,5.872605e-14
183,Zambia,2.941779e-15,2.942037e-15


In [20]:
# Function to convert country name to ISO Alpha-2 code
def get_country_code(name):
    try:
        return pyc.countries.lookup(name).alpha_2
    except LookupError:
        try:
            return extra_mappings[name]
        except LookupError:
            print("Country code not found for ", name)
            return None  # Return None if country not found

extra_mappings = {
    "Turkey": "TR",
    "Russia": "RU",
    "Bahamas, The": "BS",
    "Brunei": "BN",
    "Cape Verde": "CV",
    "Congo DRC": "CD",
    "Falkland Islands (Islas Malvinas)": "FK",
    "Gambia, The": "GM",
    "Ivory Coast": "CI",
    "Macedonia": "MK",
    "Myanmar (Burma)": "MM",
    "Reunion": "RE",
    "St. Vincent and the Grenadines": "VC",
    "Svalbard": "SJ",
    "Swaziland": "SZ",
    "Virgin Islands": "VG",
    "Western Samoa": "WS",
}

lci_water["Country_Code"] = lci_water["Country"].apply(get_country_code)
# remove rows with missing country codes
lci_water = lci_water.dropna(subset=["Country_Code"])
# malta is missing from lc-impact, add malta as new row with country code MT and EU averages
cf_certain_eu = 4.57E-15
cf_all_eu = 9.02E-15
row = pd.DataFrame({
    "Country": ["Malta"],
    "CF certain effects [PDF·yr/m3]": [cf_certain_eu],
    "CF all effects  [PDF·yr/m3]": [cf_all_eu],
    "Country_Code": ["MT"],
})
lci_water = pd.concat([lci_water, row], ignore_index=True)
lci_water.set_index("Country_Code", inplace=True)
lci_water

Country code not found for  West Bank


,Country,CF certain effects [PDF·yr/m3],CF all effects [PDF·yr/m3]
Country_Code,,,
AF,Afghanistan,1.566105e-14,1.567708e-14
AL,Albania,3.151290e-15,3.239912e-15
DZ,Algeria,4.926726e-14,7.703947e-14
AO,Angola,2.998786e-15,4.276781e-15
AR,Argentina,2.518376e-15,2.634408e-15
...,...,...,...
WS,Western Samoa,0.000000e+00,0.000000e+00
YE,Yemen,5.808963e-14,5.872605e-14
ZM,Zambia,2.941779e-15,2.942037e-15


In [21]:
# find if some regions are missing from lci
missing_regions = [region for region in all_regions if region not in lci_water.index.unique()]
print("Missing regions (lc-impact): ", missing_regions)

# find out if some regions are missing from augmented exiobase regions 
lci_water_regions = lci_water.index
missing_from_all_regions = [region for region in lci_water_regions if region not in all_regions]
print("Missing regions (exiobase, augmented): ", missing_from_all_regions)

# assert that all regions in lci-water are in the all_regions list
assert sorted(lci_water_regions) == sorted(all_regions), "Some regions in lci-water are not in the all_regions list"

Missing regions (lc-impact):  []
Missing regions (exiobase, augmented):  []


In [22]:
# augment dr_s to create dr_u

# new regions are calculated by dividing their corresponding row region by the number of countries in the row region
# for example, row region Argentina is sub-matrix WA divided by the number of countries in row region WA
wl = dr_s.loc["WL"].copy()
wl = wl / len(row_american_countries)

we = dr_s.loc["WE"].copy()
we = we / len(row_eu_countries)

wa = dr_s.loc["WA"].copy()
wa = wa / len(row_asia_pacific_countries)

wf = dr_s.loc["WF"].copy()
wf = wf / len(row_african_countries)

wm = dr_s.loc["WM"].copy()
wm = wm / len(row_middle_eastern_countries)

dr_u = dr_s.copy()
dr_u = dr_u.drop(index=row_regions.keys(), level='region')

# build a mapping of country codes to region dataframes
country_to_region = {}
for region in all_row_region_keys:
    if region in row_eu_countries:
        country_to_region[region] = we
    elif region in row_asia_pacific_countries:
        country_to_region[region] = wa
    elif region in row_african_countries:
        country_to_region[region] = wf
    elif region in row_american_countries:
        country_to_region[region] = wl
    elif region in row_middle_eastern_countries:
        country_to_region[region] = wm
    else:
        raise ValueError(f"Unknown region: {region}")

# add all new regions to dr_u
all_indices = []
all_data = []
for region in all_row_region_keys:
    region_data = country_to_region[region].copy()
    idx = pd.MultiIndex.from_product([[region],region_data.index], names=['region', 'sector'])
    all_indices.append(idx)
    all_data.append(region_data)

combined_idx = pd.MultiIndex.from_tuples(
    [idx for subidx in all_indices for idx in subidx]
)

combined_data = pd.concat(all_data)
combined_data.index = combined_idx

dr_u = pd.concat([dr_u, combined_data])

## Calculate DR factor
Calculate the impact factors of the driver for each impact region i driven by consumption in region j product sector k.

These impact factors tell how

To calculate DR factor:
1. Calculate the monetary impact factor impact/€ from 2019 exiobase data
2. Multiply each column of DR unit with the impact factors of consumption region j in product sector k. The resulting matrix represents the distribution of impacts of 1€ consumption in each consumption region.

In [30]:
# aggregate all blue water consumption related drivers
groups = exio3_19.satellite.get_index(as_dict=True, grouping_pattern={"Water Consumption Blue.*": "Water Consumption Blue -– Total"})

exio3_19.satellite_agg = exio3_19.satellite.copy(new_name="Aggregated blue water consumption accounts")

for df_name, df in zip(exio3_19.satellite_agg.get_DataFrame(data=False, with_unit=True, with_population=False),
exio3_19.satellite_agg.get_DataFrame(data=True, with_unit=True, with_population=False)):
    if df_name == "unit":
        exio3_19.satellite_agg.__dict__[df_name] = df.groupby(groups).apply(lambda x: " & ".join(x.unit.unique()))
    else:
        exio3_19.satellite_agg.__dict__[df_name] = df.groupby(groups).sum()

In [31]:
# use 2019 impact factors for calculating dr_f
# calculate dr_f - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k

dr_f = dr_u.copy()
blue_water_total = exio3_19.satellite_agg.M.loc["Water Consumption Blue -– Total"]
scalars = blue_water_total.to_numpy() # multipliers for each column
print(len(scalars)) # 9800
print(dr_u.shape) # 37000 (impact region-sector pairs) * 9800 (consumption region-sector pairs)

# multiply each column of dr_u by the respective column value from exio3_19.satellite_agg.M
dr_f = dr_f * scalars # same as dr_f * diag(scalars) but more efficient with numpy broadcasting
dr_f

9800
(37000, 9800)


AT                \
                                                     Paddy rice         Wheat   
AT Paddy rice                                               0.0  0.000000e+00   
   Wheat                                                    0.0  1.848925e-07   
   Cereal grains nec                                        0.0  9.770835e-09   
   Vegetables, fruit, nuts                                  0.0  1.246918e-08   
   Oil seeds                                                0.0  2.021756e-09   
...                                                         ...           ...   
YE Membership organisation services n.e.c. (91)             0.0  0.000000e+00   
   Recreational, cultural and sporting services (92)        0.0  0.000000e+00   
   Other services (93)                                      0.0  0.000000e+00   
   Private households with employed persons (95)            0.0  0.000000e+00   
   Extra-territorial organizations and bodies               0.0  0.000000e+00   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                             0.000000e+00   
   Wheat                                                  7.296232e-12   
   Cereal grains nec                                      4.876337e-04   
   Vegetables, fruit, nuts                                2.101149e-08   
   Oil seeds                                              2.879523e-09   
...                                                                ...   
YE Membership organisation services n.e.c. (91)           0.000000e+00   
   Recreational, cultural and sporting services (92)      0.000000e+00   
   Other services (93)                                    0.000000e+00   
   Private households with employed persons (95)          0.000000e+00   
   Extra-territorial organizations and bodies             0.000000e+00   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                   0.000000e+00   
   Wheat                                                        1.354132e-11   
   Cereal grains nec                                            3.456543e-08   
   Vegetables, fruit, nuts                                      2.023524e-03   
   Oil seeds                                                    4.311647e-09   
...                                                                      ...   
YE Membership organisation services n.e.c. (91)                 0.000000e+00   
   Recreational, cultural and sporting services (92)            0.000000e+00   
   Other services (93)                                          0.000000e+00   
   Private households with employed persons (95)                0.000000e+00   
   Extra-territorial organizations and bodies                   0.000000e+00   

                                                                    \
                                                         Oil seeds   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              2.306812e-12   
   Cereal grains nec                                  4.181442e-09   
   Vegetables, fruit, nuts                            6.116777e-09   
   Oil seeds                                          1.606631e-05   
...                                                            ...   
YE Membership organisation services n.e.c. (91)       0.000000e+00   
   Recreational, cultural and sporting services (92)  0.000000e+00   
   Other services (93)                                0.000000e+00   
   Private households with employed persons (95)      0.000000e+00   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                             \
             

## Calculate BDe factors (PDF/€ for each consumption region and product sector)

1. Multiply each cell of dr_u with the CF (characterisation factor from lc-impact) of the impact region to get PDF values for every entry
2. Sum up columns to get the total PDF/€ for the consumption region j product sector k

In [32]:
# sort rows on lci_water in same order as dr_f.index.sortlevel
lci_water = lci_water.reindex(dr_f.index.get_level_values(0).unique())

# build 1x3800 array from CF all effects
# every value should be repeated 200 times (number of sectors)
cf_all_effects = lci_water["CF all effects  [PDF·yr/m3]"].to_numpy()
cf_all_effects = np.repeat(cf_all_effects, 200)
# expand cf_all_effects to match the shape of dr_f
cf_all_effects = np.tile(cf_all_effects, (dr_f.shape[1], 1)).T
print(cf_all_effects.shape) # (37000, 9800)

# multiply each column of dr_f with cf_all_effects
pdf = dr_f * cf_all_effects
pdf

(37000, 9800)


AT                \
                                                     Paddy rice         Wheat   
AT Paddy rice                                               0.0  0.000000e+00   
   Wheat                                                    0.0  6.584002e-20   
   Cereal grains nec                                        0.0  3.479384e-21   
   Vegetables, fruit, nuts                                  0.0  4.440261e-21   
   Oil seeds                                                0.0  7.199452e-22   
...                                                         ...           ...   
YE Membership organisation services n.e.c. (91)             0.0  0.000000e+00   
   Recreational, cultural and sporting services (92)        0.0  0.000000e+00   
   Other services (93)                                      0.0  0.000000e+00   
   Private households with employed persons (95)            0.0  0.000000e+00   
   Extra-territorial organizations and bodies               0.0  0.000000e+00   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                             0.000000e+00   
   Wheat                                                  2.598180e-24   
   Cereal grains nec                                      1.736458e-16   
   Vegetables, fruit, nuts                                7.482168e-21   
   Oil seeds                                              1.025395e-21   
...                                                                ...   
YE Membership organisation services n.e.c. (91)           0.000000e+00   
   Recreational, cultural and sporting services (92)      0.000000e+00   
   Other services (93)                                    0.000000e+00   
   Private households with employed persons (95)          0.000000e+00   
   Extra-territorial organizations and bodies             0.000000e+00   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                   0.000000e+00   
   Wheat                                                        4.822050e-24   
   Cereal grains nec                                            1.230871e-20   
   Vegetables, fruit, nuts                                      7.205746e-16   
   Oil seeds                                                    1.535373e-21   
...                                                                      ...   
YE Membership organisation services n.e.c. (91)                 0.000000e+00   
   Recreational, cultural and sporting services (92)            0.000000e+00   
   Other services (93)                                          0.000000e+00   
   Private households with employed persons (95)                0.000000e+00   
   Extra-territorial organizations and bodies                   0.000000e+00   

                                                                    \
                                                         Oil seeds   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              8.214533e-25   
   Cereal grains nec                                  1.489007e-21   
   Vegetables, fruit, nuts                            2.178177e-21   
   Oil seeds                                          5.721195e-18   
...                                                            ...   
YE Membership organisation services n.e.c. (91)       0.000000e+00   
   Recreational, cultural and sporting services (92)  0.000000e+00   
   Other services (93)                                0.000000e+00   
   Private households with employed persons (95)      0.000000e+00   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                             \
             

In [33]:
# sum up all columns to get the total impact for each region-sector pair
pdf_total = pdf.sum()
pdf_total

AT  Paddy rice                                           0.000000e+00
    Wheat                                                2.992033e-16
    Cereal grains nec                                    1.329208e-15
    Vegetables, fruit, nuts                              2.656792e-15
    Oil seeds                                            2.644782e-15
                                                             ...     
WM  Membership organisation services n.e.c. (91)         1.998152e-15
    Recreational, cultural and sporting services (92)    2.369260e-15
    Other services (93)                                  3.298685e-15
    Private households with employed persons (95)        1.536901e-15
    Extra-territorial organizations and bodies           0.000000e+00
Length: 9800, dtype: float64

In [34]:
pd.DataFrame(pdf_total).to_csv("csv/pdf-water.csv", index=True)